In [1]:
# Train a word2vec skip gram model over Test8 data

# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
%matplotlib inline
from __future__ import print_function
import collections
import math
import numpy as np
import os
import random
import tensorflow as tf
import zipfile
from matplotlib import pylab
from six.moves import range
from six.moves.urllib.request import urlretrieve
from sklearn.manifold import TSNE

In [2]:
# NOTE: This doesn't work. I downloaded the file manually.
# Use the utility function Download Text8.py

url = 'http://mattmahoney.net/dc/'

def maybe_download(filename, expected_bytes):
  """Download a file if not present, and make sure it's the right size."""
  if not os.path.exists(filename):
    filename, _ = urlretrieve(url + filename, filename)
  statinfo = os.stat(filename)
  if statinfo.st_size == expected_bytes:
    print('Found and verified %s' % filename)
  else:
    print(statinfo.st_size)
    raise Exception(
      'Failed to verify ' + filename + '. Can you get to it with a browser?')
  return filename

filename = maybe_download('/Users/rgparekh/Documents/Personal/Rajesh/Data/text8.zip', 31344016)

HTTPError: HTTP Error 404: Not Found

In [2]:
# Read the data

filename = "/Users/rgparekh/Documents/Personal/Rajesh/Data/text8.zip"

def read_data(filename):
  """Extract the first file enclosed in a zip file as a list of words"""
  with zipfile.ZipFile(filename) as f:
    data = tf.compat.as_str(f.read(f.namelist()[0])).split()
  return data
  
words = read_data(filename)
print('Data size %d' % len(words))

Data size 17005207


In [3]:
words[0:10]

['anarchism',
 'originated',
 'as',
 'a',
 'term',
 'of',
 'abuse',
 'first',
 'used',
 'against']

In [4]:
vocabulary_size = 50000

def build_dataset(words):
  count = [['UNK', -1]]

  # collections.Counter calculates the word frequency for each word
  # results are ordered in descending order of frequency
  count.extend(collections.Counter(words).most_common(vocabulary_size - 1))

  # dictionary of word indexes (starting with 0 for UNK)
  # UNK: 0, etc.
  dictionary = dict()
  for word, _ in count:
    dictionary[word] = len(dictionary)
    
  # list of word indexes of the same length as words
  data = list()
  unk_count = 0
  for word in words:
    if word in dictionary:
      index = dictionary[word]
    else:
      index = 0  # dictionary['UNK']
      unk_count = unk_count + 1
    data.append(index)
  count[0][1] = unk_count

  # dictionary of index : word
  # 0: UNK etc.
  reverse_dictionary = dict(zip(dictionary.values(), dictionary.keys())) 
  return data, count, dictionary, reverse_dictionary

data, count, dictionary, reverse_dictionary = build_dataset(words)
print('Most common words (+UNK)', count[:5])
print('Sample data', data[:10])
del words  # Hint to reduce memory.

Most common words (+UNK) [['UNK', 418391], ('the', 1061396), ('of', 593677), ('and', 416629), ('one', 411764)]
Sample data [5242, 3084, 12, 6, 195, 2, 3137, 46, 59, 156]


In [6]:
# Function to generate training data
# Idea: Generate a buffer (dequeue) from the data. Use center of the buffer as the target.
# The batch will include the center of the buffer as the word and randomly chosen words from
# the skip_window on either side as labels

# e.g., Data: ['Soccer', 'is', 'extremely', 'popular', 'in', 'Europe', 'and', 'Latin', 'America']
# num_skips = 4 and skip_window = 2
# buffer size will be 5. buffer = ['Soccer', 'is', 'extremely', 'popular', 'in']
# The first batch element is 'extremely'
# labels will be randomly chosen from ['Soccer', 'is', 'popular', 'in']
# So we generate batch examples like:
# batch: ['extremely', 'extremely', 'extremely', 'extremely', 'popular', 'popular', 'popular', 'popular']
# labels: ['in', 'Soccer', 'is', 'popular', 'extremely', 'Europe', 'in', 'is']

data_index = 0

def generate_batch(batch_size, num_skips, skip_window):
  global data_index
  assert batch_size % num_skips == 0
  assert num_skips <= 2 * skip_window
  batch = np.ndarray(shape=(batch_size), dtype=np.int32)
  print('batch = ', batch)
  labels = np.ndarray(shape=(batch_size, 1), dtype=np.int32) #2D array batch_size X 1
  print('labels = ', labels)
  span = 2 * skip_window + 1 # [ skip_window target skip_window ]
  print('span: ', span)
  buffer = collections.deque(maxlen=span)
  print('buffer: ', buffer)
  for _ in range(span):
    buffer.append(data[data_index])
    data_index = (data_index + 1) % len(data)
  print('buffer: ', buffer)
  print('data_index: ', data_index)
  print(batch_size // num_skips)
  for i in range(batch_size // num_skips):
    target = skip_window  # target label at the center of the buffer
    targets_to_avoid = [ skip_window ]
    print('targets_to_avoid: ', targets_to_avoid)
    for j in range(num_skips):
      while target in targets_to_avoid:
        target = random.randint(0, span - 1)
      targets_to_avoid.append(target)
      batch[i * num_skips + j] = buffer[skip_window]
      labels[i * num_skips + j, 0] = buffer[target]
      print ('i: %d, j: %d' % (i,j))
      print('target: ', target)
      print('targets_to_avoid: ', targets_to_avoid)
      print ('batch = ', batch)
      print ('labels = ', labels)
    buffer.append(data[data_index])
    data_index = (data_index + 1) % len(data)
    print('buffer: ', buffer)
    print('data_index: ', data_index)
  return batch, labels

print('data:', [di for di in data[:32]])
print('r data:', [reverse_dictionary[di] for di in data[:32]])


for num_skips, skip_window in [(2, 1), (4, 2)]:
    data_index = 0
    batch, labels = generate_batch(batch_size=8, num_skips=num_skips, skip_window=skip_window)
    print('\nwith num_skips = %d and skip_window = %d:' % (num_skips, skip_window))
    print('    batch:', [reverse_dictionary[bi] for bi in batch])
    print('    labels:', [reverse_dictionary[li] for li in labels.reshape(8)])
    

data: ['anarchism', 'originated', 'as', 'a', 'term', 'of', 'abuse', 'first']

with num_skips = 2 and skip_window = 1:
    batch: ['originated', 'originated', 'as', 'as', 'a', 'a', 'term', 'term']
    labels: ['anarchism', 'as', 'a', 'originated', 'as', 'term', 'of', 'a']

with num_skips = 4 and skip_window = 2:
    batch: ['as', 'as', 'as', 'as', 'a', 'a', 'a', 'a']
    labels: ['anarchism', 'term', 'originated', 'a', 'of', 'term', 'originated', 'as']


In [7]:
# Training

batch_size = 128
embedding_size = 128 # Dimension of the embedding vector.
skip_window = 1 # How many words to consider left and right.
num_skips = 2 # How many times to reuse an input to generate a label.
# We pick a random validation set to sample nearest neighbors. here we limit the
# validation samples to the words that have a low numeric ID, which by
# construction are also the most frequent. 
valid_size = 128 # Random set of words to evaluate similarity on.
valid_window = 500 # Only pick few samples in the head of the distribution.
valid_examples = np.array(random.sample(range(valid_window), valid_size))
num_sampled = 128 # Number of negative examples to sample.

graph = tf.Graph()

with graph.as_default(), tf.device('/cpu:0'):

  # Input data.
  train_dataset = tf.placeholder(tf.int32, shape=[batch_size])
  train_labels = tf.placeholder(tf.int32, shape=[batch_size, 1])
  valid_dataset = tf.constant(valid_examples, dtype=tf.int32)
  
  # Variables.
  embeddings = tf.Variable(
    tf.random_uniform([vocabulary_size, embedding_size], -1.0, 1.0))
  softmax_weights = tf.Variable(
    tf.truncated_normal([vocabulary_size, embedding_size],
                         stddev=1.0 / math.sqrt(embedding_size)))
  softmax_biases = tf.Variable(tf.zeros([vocabulary_size]))
  
  # Model.
  # Look up embeddings for inputs.
  embed = tf.nn.embedding_lookup(embeddings, train_dataset)
  # Compute the softmax loss, using a sample of the negative labels each time.
  loss = tf.reduce_mean(
    tf.nn.sampled_softmax_loss(weights=softmax_weights, biases=softmax_biases, inputs=embed,
                               labels=train_labels, num_sampled=num_sampled, num_classes=vocabulary_size))

  # Optimizer.
  # Note: The optimizer will optimize the softmax_weights AND the embeddings.
  # This is because the embeddings are defined as a variable quantity and the
  # optimizer's `minimize` method will by default modify all variable quantities 
  # that contribute to the tensor it is passed.
  # See docs on `tf.train.Optimizer.minimize()` for more details.
  optimizer = tf.train.AdagradOptimizer(1.0).minimize(loss)
  
  # Compute the similarity between minibatch examples and all embeddings.
  # We use the cosine distance:
  norm = tf.sqrt(tf.reduce_sum(tf.square(embeddings), 1, keep_dims=True))
  normalized_embeddings = embeddings / norm
  valid_embeddings = tf.nn.embedding_lookup(
    normalized_embeddings, valid_dataset)
  similarity = tf.matmul(valid_embeddings, tf.transpose(normalized_embeddings))


# Iterations
num_steps = 100001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  average_loss = 0
  for step in range(num_steps):
    batch_data, batch_labels = generate_batch(
      batch_size, num_skips, skip_window)
    feed_dict = {train_dataset : batch_data, train_labels : batch_labels}
    _, l = session.run([optimizer, loss], feed_dict=feed_dict)
    average_loss += l
    if step % 5000 == 0:
      if step > 0:
        average_loss = average_loss / 2000
      # The average loss is an estimate of the loss over the last 2000 batches.
      print('Average loss at step %d: %f' % (step, average_loss))
      average_loss = 0
    # note that this is expensive (~20% slowdown if computed every 500 steps)
    if step % 20000 == 0:
      sim = similarity.eval()
      for i in range(valid_size):
        valid_word = reverse_dictionary[valid_examples[i]]
        top_k = 8 # number of nearest neighbors
        nearest = (-sim[i, :]).argsort()[1:top_k+1]
        log = 'Nearest to %s:' % valid_word
        for k in range(top_k):
          close_word = reverse_dictionary[nearest[k]]
          log = '%s %s,' % (log, close_word)
        print(log)
  final_embeddings = normalized_embeddings.eval()

Initialized
Average loss at step 0: 8.389966
Nearest to today: visigoth, centaur, emphasize, spacetime, papyri, viking, irving, coppola,
Nearest to th: already, lymphedema, mutilated, pauline, combine, brunswick, warfare, iconostasis,
Nearest to german: dawson, raise, acidosis, occasional, verrocchio, alice, circularly, categorized,
Nearest to play: orange, candid, mystic, speculate, pdpa, cemetery, felons, scarab,
Nearest to line: alicante, minded, micromachining, intentional, childress, anymore, latvia, judiciary,
Nearest to book: zagreb, picatinny, lms, millennialism, sunderland, minibuses, lemay, undetermined,
Nearest to history: porphyry, circulatory, secretory, markham, mtv, bs, phoca, writeups,
Nearest to black: hardwicke, putin, thunderbird, fictional, devouring, texans, seafaring, geniuses,
Nearest to often: fontane, contacting, kuna, lempel, returner, planck, autonegotiation, paperback,
Nearest to also: appreciative, cavalrymen, grumman, pentagonal, braun, parkway, involuntar

Nearest to position: interest, wedge, entrepreneurial, kolbin, muskegon, contacting, earl, compaor,
Nearest to made: dispelled, ilm, recherche, skimmed, carbines, parmenides, baraka, economists,
Nearest to various: marek, balaguer, opportunity, umlauts, ducas, romantics, tariq, conurbation,
Average loss at step 5000: 11.470949
Average loss at step 10000: 10.545754
Average loss at step 15000: 10.220415
Average loss at step 20000: 10.001427
Nearest to today: ars, exchange, visigoth, gerda, emphasize, positive, centaur, sanderson,
Nearest to th: four, bc, rashid, lymphedema, mutilated, rd, dementia, zinoviev,
Nearest to german: french, dawson, swedish, categorized, italian, circularly, gdansk, berthe,
Nearest to play: speculate, universal, thermionic, sunglasses, conscript, floorball, boroughs, mystic,
Nearest to line: kos, wounded, minded, merril, moli, embody, grooved, alicante,
Nearest to book: dpp, picatinny, larps, drainage, wyclif, lack, millennialism, minibuses,
Nearest to history:

Average loss at step 25000: 10.059922
Average loss at step 30000: 10.068564
Average loss at step 35000: 10.081922
Average loss at step 40000: 9.760886
Nearest to today: ars, exchange, synchronize, positive, visigoth, vicente, gerda, door,
Nearest to th: rd, bc, five, eight, six, commedia, donohue, seven,
Nearest to german: french, swedish, berthe, federal, japanese, italian, american, circularly,
Nearest to play: home, speculate, order, gting, side, sunglasses, orange, universal,
Nearest to line: moli, wounded, embody, merril, minded, grooved, anymore, printer,
Nearest to book: dpp, larps, minimized, earthbound, picatinny, drainage, mph, wyclif,
Nearest to history: art, politics, plucked, brazil, science, jit, encyclopedia, lowers,
Nearest to black: hardwicke, mysterious, narbonne, texans, fictional, devouring, grizzly, ergative,
Nearest to often: sometimes, still, usually, also, generally, now, frequently, who,
Nearest to also: often, sometimes, now, which, still, who, there, not,
Nea

Average loss at step 45000: 9.968474
Average loss at step 50000: 9.783813
Average loss at step 55000: 9.935075
Average loss at step 60000: 9.864627
Nearest to today: currently, synchronize, ars, there, exchange, settlers, now, understanding,
Nearest to th: bc, rd, seven, four, eight, six, nd, five,
Nearest to german: french, british, japanese, swedish, polish, italian, russian, federal,
Nearest to play: home, game, libri, useful, speculate, reference, gting, printer,
Nearest to line: lines, wounded, moli, cochin, strategy, kara, merril, memphis,
Nearest to book: earthbound, larps, encoding, collection, fulfilling, story, minimized, origination,
Nearest to history: politics, list, art, brazil, encyclopedia, development, science, economy,
Nearest to black: grizzly, narbonne, mysterious, hardwicke, fictional, meaningful, reznor, tolerances,
Nearest to often: sometimes, usually, generally, frequently, commonly, now, also, still,
Nearest to also: now, often, still, sometimes, which, never, 

Nearest to early: late, hillery, octave, sine, sensor, important, papua, kapitan,
Nearest to own: bbn, ghaznavid, retroactive, welland, gracie, mother, grace, stimulating,
Nearest to should: may, would, must, could, will, can, might, cannot,
Nearest to generally: often, usually, sometimes, now, there, still, widely, also,
Nearest to s: trypanosomiasis, whose, mudd, isbn, anyway, atmospheres, joseph, daoud,
Nearest to less: more, very, greater, rather, myrna, most, angiosperm, inferred,
Nearest to position: interest, opinion, muskegon, goldbach, bankrupt, percent, earl, diocese,
Nearest to made: set, progs, commanded, cited, elric, created, insect, carried,
Nearest to various: many, several, other, different, these, all, some, factorizations,
Average loss at step 65000: 9.523853
Average loss at step 70000: 9.799002
Average loss at step 75000: 9.725211
Average loss at step 80000: 9.720993
Nearest to today: currently, now, scrabble, ars, refinement, mortis, synchronize, understanding,
Nea

Average loss at step 85000: 9.876396
Average loss at step 90000: 9.759377
Average loss at step 95000: 9.656910
Average loss at step 100000: 9.646316
Nearest to today: currently, now, still, settlers, understanding, scrabble, athens, slingshot,
Nearest to th: bc, rd, nd, veitch, donohue, second, bce, fifth,
Nearest to german: french, russian, dutch, japanese, british, english, italian, swedish,
Nearest to play: player, game, hand, psychic, test, layer, asteroid, construct,
Nearest to line: lines, wounded, memphis, end, moli, strategy, cochin, zoltan,
Nearest to book: story, books, consternation, collection, earthbound, fables, encoding, series,
Nearest to history: list, cincinnati, plucked, prevalence, art, encyclopedia, discussion, payloads,
Nearest to black: white, blue, red, narbonne, grizzly, mysterious, mediawiki, yellow,
Nearest to often: usually, sometimes, generally, commonly, frequently, typically, now, rarely,
Nearest to also: still, actually, often, never, now, below, sometim